In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
from hyperopt import hp, tpe, fmin, Trials
from tqdm import tqdm

import datetime
import copy
import json

import sys
sys.path.append('../../')

from data.dataloader import Covid19IndiaLoader
from data.processing import get_data

from models.seir import SEIR_Testing, SEIRHD, SEIR_Movement, SEIR_Movement_Testing, SEIRHD_Severity

from main.seir.fitting import single_fitting_cycle, get_variable_param_ranges
from main.seir.forecast import get_forecast, forecast_all_trials, trials_to_df, create_region_csv, create_all_csvs, write_csv
from main.seir.sensitivity import calculate_sensitivity_and_plot
from utils.create_report import create_report
from utils.enums import Columns
from utils.loss import Loss_Calculator
from viz import plot_forecast, plot_top_k_trials

In [ ]:
predictions_dict = {}

## Params

In [ ]:
date_of_interest = '2020-07-31'
forecast_days = 45
now = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
folder = str(now)
ktrials = 10
model=SEIRHD_Severity

## Select Districts to fit on

In [ ]:
state, district = ('Maharashtra', 'Mumbai')

## Get Variable Params

In [ ]:
variable_param_ranges = {
    'lockdown_R0': (1, 1.15),
    'T_inc': (4, 5),
    'T_inf': (3, 4),
    'T_recov_mild': (20, 30),
    'T_recov_moderate': (20, 30),
    'T_recov_severe': (0, 40),
    'P_moderate': (0, 0.5),
    'P_severe': (0, 0.1),
    'P_fatal': (0, 0.1),
    'E_hosp_ratio': (0, 2),
    'I_hosp_ratio': (0, 1)
}
variable_param_ranges = get_variable_param_ranges(variable_param_ranges)

## Perform M1 and M2 fits

In [ ]:
predictions_dict['m1'] = single_fitting_cycle(
    state, district, data_from_tracker=False, granular_data=True, filename=None, #Data
    model=SEIRHD_Severity, variable_param_ranges=variable_param_ranges, #Choose Model and Ranges
    train_period=7, val_period=7, num_evals=1000, initialisation='intermediate', #Optimisation related parameters
    which_compartments=['total_infected', 'hospitalised', 'recovered', 'deceased', 'stable_asymptomatic', 'stable_symptomatic', 'critical'], #Compartments to Apply Loss on 
    smooth_jump=True) #Smoothing

predictions_dict['m2'] = single_fitting_cycle(
    state, district, data_from_tracker=False, granular_data=True, filename=None, #Data
    model=SEIRHD_Severity, variable_param_ranges=variable_param_ranges, #Choose Model and Ranges
    train_period=7, val_period=0, num_evals=1000, initialisation='intermediate', #Optimisation related parameters
    which_compartments=['total_infected', 'hospitalised', 'recovered', 'deceased', 'stable_asymptomatic', 'stable_symptomatic', 'critical'], #Compartments to Apply Loss on 
    smooth_jump=True) #Smoothing
    
predictions_dict['state'] = state
predictions_dict['dist'] = district
predictions_dict['fitting_date'] = datetime.datetime.now().strftime("%Y-%m-%d")
predictions_dict['datasource'] = 'covid19api' if predictions_dict['m1']['data_from_tracker'] else 'municipality'
predictions_dict['variable_param_ranges'] = predictions_dict['m1']['variable_param_ranges']
predictions_dict['data_last_date'] = predictions_dict['m1']['data_last_date']

In [ ]:
predictions_dict['m1']['best_params']

In [ ]:
predictions_dict['m2']['best_params']

## Create Master Loss Dataframe

### M1 Loss DataFrame

In [ ]:
predictions_dict['m1']['df_loss']

### M2 Loss DataFrame

In [ ]:
predictions_dict['m2']['df_loss']

## Sensitivity Plot

In [ ]:
var_tuples = [
    ('lockdown_R0', np.linspace(1, 1.5, 101), 'total_infected', None),
    ('I_hosp_ratio', np.linspace(0, 1, 201), 'total_infected', None),
    ('E_hosp_ratio', np.linspace(0, 2, 201), 'total_infected', None),
    ('P_moderate', np.linspace(0, 0.5, 201), 'stable_symptomatic', 'total_infected'),
    ('P_severe', np.linspace(0, 0.5, 201), 'critical', 'total_infected'),
    ('P_fatal', np.linspace(0, 0.5, 201), 'deceased', 'total_infected'),
    ('T_recov_mild', np.linspace(1, 100, 101), 'stable_asymptomatic', 'total_infected'),
    ('T_recov_moderate', np.linspace(1, 100, 101), 'stable_symptomatic', 'total_infected'),
    ('T_recov_severe', np.linspace(1, 100, 101), 'critical', 'total_infected'),
    ('T_recov_fatal', np.linspace(1, 100, 101), 'deceased', 'total_infected')
]

predictions_dict['m1']['plots']['sensitivity'], _, _ = calculate_sensitivity_and_plot(predictions_dict, which_fit='m1', var_tuples=var_tuples)
predictions_dict['m2']['plots']['sensitivity'], _, _ = calculate_sensitivity_and_plot(predictions_dict, which_fit='m2', var_tuples=var_tuples)

## Plot Forecasts

In [ ]:
predictions_dict['m2']['forecasts'] = {}
predictions_dict['m2']['forecasts']['best'] = get_forecast(predictions_dict, train_fit='m2', days=forecast_days)
predictions_dict['m2']['plots']['forecast_best'] = plot_forecast(predictions_dict, (state, district), error_bars=True)

trials_processed = forecast_all_trials(predictions_dict, train_fit='m1', forecast_days=forecast_days)
predictions_dict['m1']['trials_processed'] = trials_processed
predictions_dict['m1']['all_trials'] = trials_to_df(trials_processed, column=Columns.confirmed)
trials_processed = forecast_all_trials(predictions_dict, train_fit='m2', forecast_days=forecast_days)
predictions_dict['m2']['trials_processed'] = trials_processed
predictions_dict['m2']['all_trials'] = trials_to_df(trials_processed, column=Columns.confirmed)
kforecasts = plot_top_k_trials(
    predictions_dict,
    train_fit='m2',
    trials_processed=predictions_dict['m2']['trials_processed'], 
    k=ktrials,
    which_compartments=[Columns.confirmed, Columns.active, Columns.stable_symptomatic, Columns.stable_asymptomatic])
predictions_dict['m2']['plots']['forecast_confirmed_topk'] = kforecasts[Columns.confirmed]
predictions_dict['m2']['plots']['forecast_active_topk'] = kforecasts[Columns.active]
